In [1]:
%load_ext autoreload

In [2]:
autoreload 2

In [3]:
# Import Libraries

import math
import pandas as pd
import os
import torch.optim as optim
from sklearn import preprocessing
from tqdm import tqdm_notebook as tqdm
import numpy as np
import torch
import torch.nn as nn

import sys
sys.path.append('../')
sys.path.append('../BayesCNN/')
from utils import DEVICE
from torch.autograd import Variable

In [4]:
#### CUDA NOT YET IMPLEMENTED - DISABLE IN BayesBackpropagation.py ###

NB_STEPS = 10000
print('running for {0} steps'.format(NB_STEPS))

# Import data from file
df = pd.read_csv(os.getcwd() + '/Pytorch/agaricus-lepiota.data', sep=',', header=None,
             error_bad_lines=False, warn_bad_lines=True, low_memory=False)

# Set pandas to output all of the columns in output
df.columns = ['class','cap-shape','cap-surface','cap-color','bruises','odor','gill-attachment',
         'gill-spacing','gill-size','gill-color','stalk-shape','stalk-root',
         'stalk-surf-above-ring','stalk-surf-below-ring','stalk-color-above-ring','stalk-color-below-ring',
         'veil-type','veil-color','ring-number','ring-type','spore-color','population','habitat']

# Split context from label
X = pd.DataFrame(df, columns=df.columns[1:len(df.columns)], index=df.index)
# Put the class values (0th column) into Y
Y = df['class']

# Transform labels into one-hot encoded array
le = preprocessing.LabelEncoder()
le.fit(Y)
y = le.transform(Y)

# Temporary variable to avoid error 
x_tmp = pd.DataFrame(X,columns=[X.columns[0]])

# Encode each feature column and add it to x_train 
for colname in X.columns:
    le.fit(X[colname])
    #print(colname, le.classes_)
    x_tmp[colname] = le.transform(X[colname])

# Produce mushroom array: 8124 mushrooms, each with 117 one-hot encoded features
oh = preprocessing.OneHotEncoder(categorical_features='all')
oh.fit(x_tmp)
x = oh.transform(x_tmp).toarray()

running for 10000 steps


/data/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/data/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:375: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. The passed value of 'all' is the default and can simply be removed.
  DeprecationWarning)


In [5]:
def get_reward(eaten, edible):
    # REWARDS FOR AGENT
    #  Eat poisonous mushroom
    if not eaten:
        return 0
    if eaten and edible:
        return 5
    elif eaten and not edible:
        return 5 if np.random.rand() > 0.5 else -35

def oracle_reward(edible):
    return 5*edible    

def init_buffer():
    bufferX, bufferY = [], []
    for i in np.random.choice(range(len(x)), 4096):
        eat = np.random.rand()>0.5
        bufferX.append(np.concatenate((x[i], [1, 0] if eat else [0, 1])))
        bufferY.append(get_reward(eat, y[i]))
    return bufferX, bufferY

# Define some hyperparameters

Var = lambda x, dtype=torch.FloatTensor: Variable(
    torch.from_numpy(x).type(dtype)).to(DEVICE)

bufferX, bufferY = init_buffer()

In [6]:
class MushroomNet():
    def __init__(self, label = 'MushNet', n_weight_sampling=2):
        self.label = label
        self.n_weight_sampling = n_weight_sampling
        self.epsilon = 0
        self.net = None
        self.loss, self.optimizer = None, None
        self.cum_regrets = [0]
        self.bufferX = np.copy(bufferX)
        self.bufferY = np.copy(bufferY)
    
    # Use NN to decide next action
    def try_ (self, mushroom):
        samples = self.n_weight_sampling
        context, edible = x[mushroom], y[mushroom]
        try_eat = Var(np.concatenate((context, [1, 0])))
        try_reject = Var(np.concatenate((context, [0, 1])))
        
        # Calculate rewards using model
        with torch.no_grad():
            r_eat = sum([self.net(try_eat) for _ in range(samples)]).item()/samples
            r_reject = sum([self.net(try_reject) for _ in range(samples)]).item()/samples
        
        # Take random action for epsilon greedy agents, calculate agent's reward
        eaten = r_eat > r_reject
        if np.random.rand()<self.epsilon:
            eaten = (np.random.rand()<.5)
        agent_reward = get_reward(eaten, edible)
        
        # Get rewards and update buffer
        if eaten:
            action = [1, 0]
        else:
            action = [0, 1]
        bufferX.append(np.concatenate((context, action)))
        bufferY.append(agent_reward)
        
        # Calculate regret
        oracle = oracle_reward(edible)
        regret = oracle - agent_reward
        self.cum_regrets.append(self.cum_regrets[-1]+regret)
        return regret

    # Feed next mushroom
    def update(self, mushroom):
        rg = self.try_(mushroom)
        bX = Var(np.array(self.bufferX[-4096:]))
        bY = Var(np.array(self.bufferY[-4096:]))
        for idx in np.split(np.random.permutation(range(4096)), 64):
            self.loss_step(bX[idx], bY[idx])
        return rg
    
    def loss_step(self, x, y):
        raise NotImplementedError
            

In [7]:
# Class for BBB agent
from bayes import BayesWrapper
from utils import prior_nll

mlp = lambda inputs:nn.Sequential(
        nn.Linear(inputs, 100), nn.ReLU(),
        nn.Linear(100, 100), nn.ReLU(),
        nn.Linear(100, 1)).to(DEVICE)

class BBB_MNet(MushroomNet):
    def __init__(self, label):
        super().__init__(label)
        self.net = BayesWrapper(name='BayesMush',
                               net = mlp(x.shape[1]+2),
                               prior_nll=prior_nll,
                               type='regression')

    def loss_step(self, x, y, n_samples = 2):
        outputs = [self.net(x)[:,0] for _ in range(n_samples)]
        self.net.step(outputs, y, 1/64)


# Class for Greedy agents
class EpsGreedyMlp(MushroomNet):
    def __init__(self, epsilon=0, **kwargs):
        super().__init__(**kwargs)
        self.n_weight_sampling = 1
        self.epsilon = epsilon
        self.net = mlp(x.shape[1]+2)
        self.bufferX, self.bufferY = init_buffer()
        self.optimizer = optim.SGD(self.net.parameters(), lr = 0.001)
        self.mse = nn.MSELoss()
        
    def loss_step(self, x, y):
        self.net.zero_grad()
        loss = self.mse(self.net.forward(x), y)
        loss.backward()
        self.optimizer.step()

In [ ]:
NB_STEPS = 2000

mushroom_nets = {'bbb':BBB_MNet(label = 'BBB'),
    'e0':EpsGreedyMlp(epsilon=0, label = 'Greedy')}


for step in range(NB_STEPS):
    mushroom = np.random.randint(len(x))
    for j, (key, net) in enumerate(mushroom_nets.items()):
        net.update(mushroom)
    print('\r Step {2}/{3} BBB : {0}, Greedy: {1}'.format(
        mushroom_nets['bbb'].cum_regrets[-1],
        mushroom_nets['e0'].cum_regrets[-1], step+1, NB_STEPS), end='')

 Step 1555/2000 BBB : 3190, Greedy: 1835

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict({net.label: net.cum_regrets for i, net in mushroom_nets.items()})
df.to_csv('mushroom_regrets_bbb.csv')

In [ ]:
Step 1100: 
me : BBB= 2300, Greedy=1600
Yours : Greedy = 2600, BBB = 800